<a href="https://colab.research.google.com/github/Silverdroplet/HueDetect/blob/main/Hackathon_HueDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

In [ ]:
!pip install webcolors

In [ ]:
# William's Token for ngrok
!ngrok config add-authtoken 2nfHsvQZCweW9K3dNEjZiSqtrDx_2z7CQuERVscStWEgLn2mj


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install webcolors==1.13

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from PIL import Image
import numpy as np
from collections import Counter
from sklearn.cluster import KMeans
import webcolors
import re
import base64
from io import BytesIO

app = Flask(__name__)

# Function to convert RGB to a human-readable color name and format it
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    closest_name = min_colors[min(min_colors.keys())]
    formatted_name = re.sub(r"([a-z])([A-Z])", r"\1 \2", closest_name)
    return formatted_name.title()

# Function to detect the dominant color
def detect_dominant_color(image, n_colors=3):
    image = image.convert("RGB")
    image = image.resize((100, 100))  # Resize for faster processing
    image_array = np.array(image).reshape((-1, 3))
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(image_array)
    colors = kmeans.cluster_centers_
    counts = Counter(kmeans.labels_)
    dominant_color = colors[counts.most_common(1)[0][0]]
    return dominant_color, colors

# Serve HTML directly in the route
@app.route('/')
def home():
    return '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HueDetects</title>
    <style>
        /* Global styles */
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f0f2f5;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            background: linear-gradient(135deg, #ffafbd, #ffc3a0);
        }

        .container {
            background-color: white;
            padding: 40px;
            border-radius: 16px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.2);
            max-width: 600px;
            width: 100%;
            text-align: center;
            transition: transform 0.3s ease;
        }

        .container:hover {
            transform: translateY(-5px); /* Slight hover effect */
        }

        h1 {
            font-size: 36px;
            color: #2c3e50;
            margin-bottom: 30px;
        }

        .file-input, .file-input:disabled {
            margin: 20px 0;
            padding: 12px 20px;
            font-size: 16px;
            background-color: #3498db;
            color: white;
            border: none;
            border-radius: 6px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }

        .file-input:hover:not(:disabled) {
            background-color: #2980b9;
        }

        .file-input:disabled {
            background-color: #bdc3c7;
            cursor: not-allowed;
        }

        .result-section {
            margin-top: 30px;
            text-align: center;
        }

        .color-box {
            display: inline-block;
            width: 50px;
            height: 50px;
            margin-right: 10px;
            border-radius: 4px;
            margin-bottom: 10px;
            border: 2px solid #ecf0f1;
        }

        #result {
            margin-top: 20px;
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
        }

        #dominantColor {
            font-size: 24px;
            font-weight: bold;
            margin-bottom: 10px;
        }

        .spinner {
            display: none;
            margin: 20px auto;
            border: 4px solid rgba(0, 0, 0, 0.1);
            border-top: 4px solid #3498db;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 0.6s linear infinite;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .error {
            color: red;
            margin-top: 10px;
        }

        video, canvas {
            margin-top: 20px;
            border-radius: 12px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
        }
    </style>
</head>
<body>

<div class="container">
    <h1>HueDetects</h1>
    <form id="uploadForm" enctype="multipart/form-data">
        <input type="file" id="fileInput" name="file" accept="image/*" class="file-input">
        <button type="submit" class="file-input">Detect Colors from File</button>
    </form>

    <div>
        <button id="startCamera" class="file-input">Capture Image from Camera</button>
        <video id="video" width="400" height="300" autoplay></video>
        <button id="capture" class="file-input" style="display: none;">Capture Photo</button>
        <canvas id="canvas" width="400" height="300" style="display: none;"></canvas>
    </div>

    <div id="result" class="result-section">
        <div id="dominantColor"></div> <!-- Dominant color text goes here -->
        <div id="colorBoxes"></div> <!-- Color boxes will be displayed here -->
    </div>
    <div id="error" class="error"></div>
</div>

<script>
    const startCameraBtn = document.getElementById('startCamera');
    const video = document.getElementById('video');
    const captureBtn = document.getElementById('capture');
    const canvas = document.getElementById('canvas');
    const context = canvas.getContext('2d');

    // Start camera when button is clicked
    startCameraBtn.addEventListener('click', function () {
        navigator.mediaDevices.getUserMedia({ video: true })
            .then(function (stream) {
                video.srcObject = stream;
                captureBtn.style.display = 'block'; // Show capture button
            })
            .catch(function (error) {
                console.error("Error accessing the camera:", error);
            });
    });

    // Capture the image from the camera
    captureBtn.addEventListener('click', function () {
        context.drawImage(video, 0, 0, canvas.width, canvas.height);
        const imageDataURL = canvas.toDataURL('image/png');
        sendImageToServer(imageDataURL);
    });

    // Function to send the captured image to the server
    function sendImageToServer(imageDataURL) {
        fetch('/upload_camera', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ image: imageDataURL })
        })
        .then(response => response.json())
        .then(data => {
            displayResults(data);
        })
        .catch(error => {
            console.error('Error:', error);
        });
    }

    // Display results from server
    function displayResults(data) {
        const resultDiv = document.getElementById('result');
        const dominantColorDiv = document.getElementById('dominantColor');
        const colorBoxesDiv = document.getElementById('colorBoxes');

        dominantColorDiv.innerHTML = `<h2>Dominant Color: ${data.dominant_color}</h2>`;
        colorBoxesDiv.innerHTML = ''; // Clear previous boxes

        data.colors.forEach(color => {
            const colorBox = document.createElement('div');
            colorBox.classList.add('color-box');
            colorBox.style.backgroundColor = color;

            const colorName = document.createElement('span');
            colorName.textContent = color; // Add color name next to the box

            const colorContainer = document.createElement('div');
            colorContainer.appendChild(colorBox);
            colorContainer.appendChild(colorName);

            colorContainer.style.display = "flex"; // Align color box and name
            colorContainer.style.alignItems = "center";
            colorContainer.style.margin = "5px";

            colorBoxesDiv.appendChild(colorContainer);
        });
    }
</script>

</body>
</html>

    '''

# Route to handle image uploads from file
@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    image = Image.open(file)
    dominant_color, color_palette = detect_dominant_color(image)

    dominant_color_name = closest_color(dominant_color.astype(int))
    color_names = [closest_color(color.astype(int)) for color in color_palette]

    return jsonify({'dominant_color': dominant_color_name, 'colors': color_names})

# Route to handle camera image upload
@app.route('/upload_camera', methods=['POST'])
def upload_camera_image():
    data = request.get_json()
    if 'image' not in data:
        return jsonify({'error': 'No image data'}), 400

    # Decode base64 image
    image_data = base64.b64decode(data['image'].split(',')[1])
    image = Image.open(BytesIO(image_data))

    dominant_color, color_palette = detect_dominant_color(image)

    dominant_color_name = closest_color(dominant_color.astype(int))
    color_names = [closest_color(color.astype(int)) for color in color_palette]

    return jsonify({'dominant_color': dominant_color_name, 'colors': color_names})

if __name__ == '__main__':
    # Start ngrok
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel available at: {url}")

    # Start Flask app
    app.run()

 * ngrok tunnel available at: NgrokTunnel: "https://3f17-104-197-5-170.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Oct/2024 18:25:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Oct/2024 18:25:54] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /upload_camera [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpo

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans
import webcolors
import re

# Function to convert RGB to a human-readable color name and format it
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    closest_name = min_colors[min(min_colors.keys())]

    # Use a regex to split words that have no spaces in between
    formatted_name = re.sub(r"([a-z])([A-Z])", r"\1 \2", closest_name)

    # Capitalize the words to title case
    return formatted_name.title()

# Function to detect the dominant color
def detect_dominant_color(image_path, n_colors=5):
    # Open the image and convert to RGB
    image = Image.open(image_path)
    image = image.convert("RGB")

    # Resize image to speed up the process
    image = image.resize((100, 100))

    # Convert image into a numpy array
    image_array = np.array(image)
    image_array = image_array.reshape((-1, 3))  # Reshape the image into a 2D array of RGB values

    # Apply KMeans clustering to find dominant colors
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(image_array)

    # Get the most common color
    colors = kmeans.cluster_centers_
    counts = Counter(kmeans.labels_)

    # Get the color with the highest frequency
    dominant_color = colors[counts.most_common(1)[0][0]]

    return dominant_color, colors

# Function to show image and detected colors with readable text
def display_image_and_colors(image_path, dominant_color, colors):
    # Open and display the image
    image = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Convert dominant color to closest color name
    dominant_color_name = closest_color(dominant_color.astype(int))
    print(f"Dominant Color: {dominant_color_name}")

    # Display color palette with text
    plt.figure(figsize=(12, 2))
    for i, color in enumerate(colors):
        plt.subplot(1, len(colors), i+1)
        plt.axis('off')
        plt.imshow([[color / 255]])  # Divide by 255 to normalize RGB values

        # Get color name and display it in title case
        color_name = closest_color(color.astype(int))
        plt.title(color_name, fontsize=12)

    plt.show()

# Main execution to analyze the uploaded image
image_path = '"C:\Users\mnros\Downloads\Screenshot 2024-10-12 165824.png"'  # Path to the uploaded image

# Detect dominant color
dominant_color, color_palette = detect_dominant_color(image_path)

# Display the image and detected colors
display_image_and_colors(image_path, dominant_color, color_palette)
